Before we perform any optimization, we need a schema.
For this example, we'll use [`HrSchema`](https://calcite.apache.org/testapidocs/org/apache/calcite/test/JdbcTest.HrSchema.html) which is defined in [`JdbcTest.java`](https://github.com/apache/calcite/blob/master/core/src/test/java/org/apache/calcite/test/JdbcTest.java) in the Apache Calcite test suite.
First we start by requiring Calcite.
In this case, we also require tests JAR so we can use `HrSchema`.

In [1]:
%%loadFromPOM
<dependency>
    <groupId>org.apache.calcite</groupId>
    <artifactId>calcite-core</artifactId>
    <version>1.21.0</version>
</dependency>
<dependency>
    <groupId>org.apache.calcite</groupId>
    <artifactId>calcite-core</artifactId>
    <version>1.21.0</version>
    <classifier>tests</classifier>
    <type>test-jar</type>
</dependency>

Here we construct an instance of `HrSchema` using `CalciteAssert` which is also from Calcite's testing libraries.
In practice, you would connect to a real database or at least some source of schema information.
With the schema, we first construct a [`RelBuilder`](https://calcite.apache.org/apidocs/org/apache/calcite/tools/RelBuilder.html) that we can use to build up a tree of relational algebra operators.

In [2]:
import org.apache.calcite.jdbc.CalciteSchema;
import org.apache.calcite.schema.SchemaPlus;
import org.apache.calcite.tools.Frameworks;
import org.apache.calcite.tools.FrameworkConfig;
import org.apache.calcite.tools.RelBuilder;

import org.apache.calcite.test.CalciteAssert;

SchemaPlus rootSchema = CalciteSchema.createRootSchema(true).plus();
FrameworkConfig config = Frameworks.newConfigBuilder()
    .defaultSchema(
        CalciteAssert.addSchema(rootSchema, CalciteAssert.SchemaSpec.HR))
    .build();
RelBuilder builder = RelBuilder.create(config);

Now we build a simple query which is roughly equivalent to the SQL query `SELECT * FROM emps, depts NATURAL JOIN deptno WHERE empid=100`.
Note that we have constructed the filter so that it falls after the join.

In [3]:
import org.apache.calcite.rel.RelNode;
import org.apache.calcite.rel.RelWriter;
import org.apache.calcite.rel.externalize.RelWriterImpl;
import org.apache.calcite.rel.core.JoinRelType;

RelNode opTree = builder.scan("emps")
    .scan("depts")
    .join(JoinRelType.INNER, "deptno")
    .filter(builder.equals(builder.field("empid"), builder.literal(100)))
    .build();
    
RelWriter rw = new RelWriterImpl(new PrintWriter(System.out, true));
opTree.explain(rw);

3:LogicalFilter(condition=[=($0, 100)])
  2:LogicalJoin(condition=[=($1, $5)], joinType=[inner])
    0:LogicalTableScan(table=[[hr, emps]])
    1:LogicalTableScan(table=[[hr, depts]])


A common query optimization is pushing filters past joins, limiting the amount of data which must be joined.
Calcite implements this with [`FilterJoinRule`](https://calcite.apache.org/apidocs/org/apache/calcite/rel/rules/FilterJoinRule.html).
Calcite has two main planners which can be used for query optimization.
The first is [`HepPlanner`](https://calcite.apache.org/apidocs/org/apache/calcite/plan/hep/HepPlanner.html) which simply applies a given set of rules until they no longer apply.
We'll start by constructin a [`HepProgram`](https://calcite.apache.org/apidocs/org/apache/calcite/plan/hep/HepProgram.html) which specifies a set of rules to be applied and specify `FilterIntoJoinRule`.

In [4]:
import org.apache.calcite.plan.hep.HepProgram;
import org.apache.calcite.plan.hep.HepProgramBuilder;
import org.apache.calcite.rel.rules.FilterJoinRule;

HepProgram program = HepProgram.builder().addRuleInstance(FilterJoinRule.FILTER_ON_JOIN).build();

Now we can use this program to construct an instance of `HepPlanner` and run the optimization.

In [5]:
import org.apache.calcite.plan.hep.HepPlanner;

HepPlanner hepPlanner = new HepPlanner(program);
hepPlanner.setRoot(opTree);
hepPlanner.findBestExp().explain(rw);

13:LogicalJoin(condition=[=($1, $5)], joinType=[inner])
  10:LogicalFilter(condition=[=($0, 100)])
    0:LogicalTableScan(table=[[hr, emps]])
  1:LogicalTableScan(table=[[hr, depts]])


The second is [`VolcanoPlanner`](https://calcite.apache.org/apidocs/org/apache/calcite/plan/volcano/VolcanoPlanner.html) which attempts to apply a set of rules to minimize the expected cost of executing a query.
`VolcanoPlanner` is the default planner used by Calcite.
Note that all the operators in the tree we constructed above are *logical* operators meaning that they don't have a specific implementation.
All logical operators must be converted to physical operators before a query can actually be implemented, which is required for `VolcanoPlanner` to provide an estimated cost.
Each physical operator in Calcite has a *calling convention* which specifies how the query will actually be executed.
Since we're not working with an actual database we'll use `EnumerableConvention` which simply implements queries over collections implementing the `Enumerable` interface.

In [6]:
import org.apache.calcite.adapter.enumerable.EnumerableConvention;
import org.apache.calcite.plan.RelTraitSet;
import org.apache.calcite.plan.RelOptCluster;
import org.apache.calcite.plan.volcano.VolcanoPlanner;

RelOptCluster cluster = opTree.getCluster();
VolcanoPlanner planner = (VolcanoPlanner) cluster.getPlanner();

RelTraitSet desiredTraits = cluster.traitSet().replace(EnumerableConvention.INSTANCE);
RelNode newRoot = planner.changeTraits(opTree, desiredTraits);
planner.setRoot(newRoot);

RelNode optimized = planner.findBestExp();
optimized.explain(rw);

150:EnumerableHashJoin(condition=[=($1, $5)], joinType=[inner])
  148:EnumerableFilter(condition=[=($0, 100)])
    147:EnumerableInterpreter
      63:BindableTableScan(table=[[hr, emps]])
  149:EnumerableInterpreter
    69:BindableTableScan(table=[[hr, depts]])


`VolcanoPlanner` has a default set of rules which includes `FilterJoinRule` so we can see that in this case, the filter has also been moved before the join.
However, we also see that logical operators have been replaced with enumerable operators.
This means we can now execute the optimized query and get a result.

In [7]:
import java.sql.ResultSet;

import org.apache.calcite.tools.RelRunners;

ResultSet result = RelRunners.run(optimized).executeQuery();
int columns = result.getMetaData().getColumnCount();
while (result.next()) {
  System.out.println(result.getString(1) + " " + result.getString(7));
}

100 Sales
